# AOC 2022

Welcome to the Advent of Code 2022 !

## Basic configuration



In [ ]:
# help for aocd : https://pypi.org/project/advent-of-code-data/

#!pip install aocd

In [ ]:
import os

# replace by your login session cookie
os.environ[
    "AOC_SESSION"
] = "53616c7465645f5f544b7f45f4e71d3973f588d3fc85718685dea33e470d469bb196eb397334fc5493bfcb535104e92c8e4c0781cf5a6e0a9888e4de1aa51bed"  # your login session cookie

In [ ]:
from aocd import submit
from aocd.models import Puzzle

In [ ]:
import numpy as np
from tqdm import tqdm
import json
import typing as tp
from collections import Counter, defaultdict, deque
import math
from itertools import product
import re
import string
import matplotlib.pyplot as plt

## Day 10
https://adventofcode.com/2022/day/10
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=10)
content = puzzle.input_data.split("\n")

### Part 1

In [ ]:
def compute_signal(n,x): 
    if n%40==20: 
        return n*x
    return 0

In [ ]:
x = 1
strenghts = []
n_cycle = 0
for line in content: 
    if line=="noop": 
        n_cycle +=1
        strenghts.append(compute_signal(n_cycle, x))
    else: 
        n_cycle += 1
        strenghts.append(compute_signal(n_cycle, x))
        n_cycle += 1
        strenghts.append(compute_signal(n_cycle, x))           
        _, v = line.split(" ")
        x += int(v)
        

In [ ]:
answ = sum(strenghts)

In [ ]:
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def sprite(x):
    return (x-1,x,x+1)

In [ ]:
def draw_crt(crt, crt_pos, x): 
    line, pos = divmod(crt_pos, 40)
    crt.append(1 if pos in sprite(x) else 0)
    return crt

In [ ]:
x = 1
n_cycle = 0
crt = []
for line in content:
    if line == "noop": 
        crt = draw_crt(crt, n_cycle, x)
        n_cycle += 1
    else: 
        crt = draw_crt(crt, n_cycle, x)
        n_cycle += 1
        
        crt = draw_crt(crt, n_cycle, x)
        n_cycle += 1
        _, v = line.split(" ")
        x += int(v)

In [ ]:
crt = np.array(crt).reshape(6, 40)

In [ ]:
plt.imshow(crt)

In [ ]:
puzzle.answer_b = "EFGERURE"

## Day 9
https://adventofcode.com/2022/day/9
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=9)
content =  puzzle.input_data.split("\n")
new_content = []
for line in content: 
    d, n = line.split(" ")
    new_content.append((d, int(n)))

In [ ]:
new_content

### Part 1

In [ ]:
def add_tuple(tup1, tup2): 
    return tuple(map(lambda i, j: i + j, tup1, tup2))

In [ ]:
def move(pos, movement): 
    return add_tuple(pos, movement)

In [ ]:
def move_closer(pos1, pos2):
    diff_x = pos2[0] - pos1[0]
    diff_y = pos2[1] - pos1[1]
    
    if abs(diff_x)>1:
        move_x = math.copysign(1,diff_x) 
        move_y = math.copysign(1,diff_y) if abs(diff_y)>=1 else 0
        
    elif abs(diff_y)>1:
        move_x = math.copysign(1,diff_x) if abs(diff_x)>=1 else 0
        move_y = math.copysign(1,diff_y) 
    else:
        move_x = 0
        move_y = 0
    
    return add_tuple(pos1, (move_x, move_y))

In [ ]:
def move_head(pos_h, direction): 
    if direction == "U": 
        pos_h = move(pos_h, (0,1))
    elif direction == "D": 
        pos_h = move(pos_h, (0,-1))
    elif direction == "L":
        pos_h = move(pos_h, (-1,0))
    else: 
        pos_h = move(pos_h, (1,0))
    return pos_h

In [ ]:
pos_h = (0,0)
pos_t = (0,0)

t_positions = [pos_t]
for direction, n in new_content:
    for _ in range(n):
        pos_h = move_head(pos_h, direction)
        
        pos_t = move_closer(pos_t, pos_h)
        t_positions.append(pos_t)


In [ ]:
answ = len(set(t_positions))
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
cur_positions = [(0,0)]*10
t_positions = [(0,0)]

for direction, n in new_content: 
    for _ in range(n): 
        new_positions = [move_head(cur_positions[0], direction)]
        for i in range(len(cur_positions)-1):
            new_pos = move_closer(cur_positions[i+1], new_positions[i])
            new_positions.append(new_pos)
        t_positions.append(new_positions[-1])
        cur_positions = new_positions

In [ ]:
answ = len(set(t_positions))
answ

In [ ]:
puzzle.answer_b = answ

## Day 8
https://adventofcode.com/2022/day/8
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=8)
content = puzzle.input_data.split("\n")

In [ ]:
content =[[int(i) for i in line] for line in content]

In [ ]:
class Tree: 
    def __init__(self, x, y, array): 
        self.x = x
        self.y = y
        self.array = array
        self.h = array[y,x]
        self.on_edge = True if (x in {0, len(array[0])-1} or y in {0, len(array)-1}) else False
        self._is_seen = None
        self._scenic_score = None
        
    def check_left(self): 
        return all(h < self.h for h in self.array[self.y,:self.x])
    def check_right(self): 
        return all(h < self.h for h in self.array[self.y,self.x + 1:])
    def check_above(self): 
        return all(h < self.h for h in self.array[:self.y,self.x])
    def check_below(self): 
        return all(h < self.h for h in self.array[self.y + 1:,self.x])
    
    def is_seen(self):
        if self._is_seen is None:
            self._is_seen = True if self.on_edge else any([self.check_left(), self.check_right(), self.check_above(), self.check_below()])
        return self._is_seen

    
    def count(self, step_x, step_y): 
        cur_x = self.x+step_x
        cur_y = self.y+step_y
        count = 0
        while cur_x >= 0 and cur_x < len(self.array[0]) and cur_y >= 0 and cur_y < len(self.array):
            cur_h = self.array[cur_y, cur_x]
            count += 1
            if cur_h >= self.h: 
                break
            cur_x += step_x
            cur_y += step_y
        return count
    
    def look_left(self):
        if self.x == 0:
            return 0
        return self.count(-1, 0)
    def look_right(self):
        if self.x == len(array[0])-1:
            return 0
        return self.count(1,0)
    def look_up(self): 
        if self.y == 0:
            return 0
        return self.count(0, -1)
    def look_down(self): 
        if self.y == len(array)-1:
            return 0
        return self.count(0,1)
    
    def scenic_score(self):
        if self._scenic_score is None: 
            self._scenic_score = math.prod([self.look_left(), self.look_right(), self.look_up(), self.look_down()])
        return self._scenic_score

In [ ]:
array = np.array(content)

In [ ]:
tree_grid = [Tree(x,y,array) for y in range(len(array)) for x in range(len(array[0]))]
    

### Part 1

In [ ]:
answ = sum(t.is_seen() for t in tree_grid)

In [ ]:
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
answ = max(t.scenic_score() for t in tree_grid)
answ

In [ ]:
puzzle.answer_b = answ

## Day 7
https://adventofcode.com/2022/day/7
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=7)
content = puzzle.input_data.split("\n")

In [ ]:
class Directory: 
    def __init__(self, path:str, parent_dir:tp.Optional[str]=None): 
        self.path = path
        self.parent_dir = parent_dir
        self.subdirs = []
        self.file_weight = 0
        self._total_weight = 0
    
    def total_weight(self): 
        if self._total_weight == 0: 
            
            subweight = sum(subd.total_weight() for subd in self.subdirs) if len(self.subdirs) != 0 else 0
            self._total_weight = self.file_weight + subweight
            
        return self._total_weight
    

In [ ]:
def get_path(directory, name): 
    return os.path.join(directory.path, name) if directory is not None else name

In [ ]:
created = {}

idx = 0
cur_dir = None

for line in content:
    cur_line = line.split(" ")
    if cur_line[0]=="$" and cur_line[1]=="ls": 
        continue
    elif cur_line[0]=="$" and cur_line[1]=="cd":
        name = cur_line[2]
        path = get_path(cur_dir, name)
        if name == "..": 
            cur_dir = cur_dir.parent_dir 
        elif path in created.keys(): 
            cur_dir = created[path]
        else: 
            cur_dir = Directory(path, cur_dir)
            created[path] = cur_dir
    elif cur_line[0] == "dir": 
        name = cur_line[1]
        path = get_path(cur_dir, name)
         
        new_dir = Directory(path, cur_dir)
        created[path] = new_dir
        cur_dir.subdirs.append(new_dir)
    else: 
        cur_dir.file_weight += int(cur_line[0])

### Part 1

In [ ]:
answ = sum(curdir.total_weight() for curdir in created.values() if curdir.total_weight()<=100000)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
free = 70000000 - created["/"].total_weight()
needed = 30000000 - free
needed

In [ ]:
answ = min(curdir.total_weight() for curdir in created.values() if curdir.total_weight()>=needed)
answ

In [ ]:
puzzle.answer_b = answ

## Day 6
https://adventofcode.com/2022/day/6
### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=6)
content = puzzle.input_data

### Part 1

In [ ]:
def find_marker(content, start_idx): 
    marker = deque(content[:start_idx-1])
    idx = start_idx-1
    while True: 
        marker.append(content[idx])
        if len(set(marker)) == len(marker): 
            break
        marker.popleft()
        idx += 1

    return idx + 1


In [ ]:
puzzle.answer_a = find_marker(content, 4)

### Part 2

In [ ]:
puzzle.answer_b = find_marker(content, 14)

## Day 5
https://adventofcode.com/2022/day/5
### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=5)

In [ ]:
stacks,procedure = puzzle.input_data.split("\n\n")

In [ ]:
def prepare_stacks(stacks:list)-> list:
    stacks = stacks.split("\n")[:-1]
    transpose_stacks = [''.join(s) for s in zip(*stacks)][1::4]
        
    new_stacks = [deque(s.strip()) for s in transpose_stacks]
    for d in new_stacks:
        d.reverse()
        
    return new_stacks

In [ ]:
new_stacks = prepare_stacks(stacks)

In [ ]:
procedure = procedure.split("\n")
new_procedure = [line.strip("move ").replace(" from ",",").replace(" to ", ",") for line in procedure]
new_procedure = [[int(x) for x in line.split(",")] for line in new_procedure]

### Part 1

In [ ]:
def apply_procedure(stacks, procedure): 
    for n_move, col1, col2 in procedure: 
        for _ in range(n_move): 
            stacks[col2-1].append(stacks[col1-1].pop())
            
    return "".join(d[-1] for d in stacks)

In [ ]:
answ = apply_procedure(new_stacks, new_procedure)

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
new_stacks = prepare_stacks(stacks)

In [ ]:
def apply_procedure_2(stacks, procedure): 
    for n_move, col1, col2 in procedure: 
        substack = []
        for _ in range(n_move): 
            substack.append(stacks[col1-1].pop())
        substack.reverse()
        stacks[col2-1].extend(substack)
    return "".join(d[-1] for d in stacks)    

In [ ]:
answ = apply_procedure_2(new_stacks, new_procedure)

In [ ]:
answ

In [ ]:
puzzle.answer_b = answ

## Day 4
https://adventofcode.com/2022/day/4
### Prepare input 

In [ ]:
puzzle = Puzzle(year=2022, day=4)
content = puzzle.input_data.split("\n")

In [ ]:
content

In [ ]:
def get_int_range(str_range:str)-> list: 
    start, stop = str_range.split("-")
    return set(i for i in range(int(start), int(stop)+1))

In [ ]:
new_content = [[get_int_range(el) for el in pair.split(",")] for pair in content]

### Part 1 

In [ ]:
count = sum(r1.issubset(r2) or r2.issubset(r1) for r1,r2 in new_content)
count

In [ ]:
puzzle.answer_a = count

### Part 2

In [ ]:
count = sum(bool(r1&r2) for r1,r2 in new_content)
count

In [ ]:
puzzle.answer_b = count

## Day 3
https://adventofcode.com/2022/day/3
### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=3)
content = puzzle.input_data.split("\n")

In [ ]:
content

### Part 1

In [ ]:
def find_item(items:str)->str: 
    half = int(len(items)/2)
    comp1 = items[:half]
    comp2 = items[half:]
    
    return list(set(comp1)&set(comp2))[0]

In [ ]:
ALPHABET = string.ascii_lowercase+string.ascii_uppercase

def get_priority(item):
    return ALPHABET.rfind(item)+1

In [ ]:
answ = sum(get_priority(find_item(items)) for items in content)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def find_group_item(group:list)->str: 
    it1, it2, it3 = group
    return list(set(it1)&set(it2)&set(it3))[0]

In [ ]:
answ = sum(get_priority(find_group_item(content[i:i+3])) for i in range(0,len(content),3))
answ

In [ ]:
puzzle.answer_b = answ

## Day 2
https://adventofcode.com/2022/day/2
### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=2)
content = [line.split(" ") for line in puzzle.input_data.split("\n")]
content

### Part 1

In [ ]:
def replace_play(play:str) -> int: 
    if play in ["A", "X"]: 
        return 1
    if play in ["B", "Y"]: 
        return 2
    if play in ["C", "Z"]:
        return 3
    
def replace_pair(pair:list) -> list: 
    return [replace_play(x) for x in pair]

In [ ]:
def compute_score(pair:list) -> int: 
    p1,p2 = replace_pair(pair)
    if (p1-p2)%3 == 1:
        return p2
    if p1 == p2 : 
        return p2 + 3
    return p2 + 6  

In [ ]:
answ = sum(compute_score(pair) for pair in content)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
def compute_score_2(pair:list) -> int: 
    p1, result = pair
    p1 = replace_play(p1)
    
    if result == "X" : 
        p2 = p1-1
        score = p2 if p2 != 0 else 3
    elif result == "Y": 
        score = p1 + 3
    else: 
        p2 = p1+1
        score = p2 if p2!=4 else 1
        score += 6
    return score

In [ ]:
answ = sum(compute_score_2(pair) for pair in content)
answ

In [ ]:
puzzle.answer_b = answ

## Day 1
https://adventofcode.com/2022/day/1

### Prepare input

In [ ]:
puzzle = Puzzle(year=2022, day=1)

In [ ]:
content =[[int(x) for x in elf.split("\n")] for elf in puzzle.input_data.split("\n\n")]
content

### Part 1

In [ ]:
answ = max(sum(elf)for elf in content)
answ

In [ ]:
puzzle.answer_a = answ

### Part 2

In [ ]:
answ = sum(sorted(sum(elf) for elf in content)[-3:])
answ

In [ ]:
puzzle.answer_b = answ